# PBO learnt on several iterations and one weigth one the car on hill environment

## Define parameters

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import jax
import jax.numpy as jnp

# keys
seed = 0
key = jax.random.PRNGKey(seed)
shuffle_key, q_network_key, random_weights_key, pbo_network_key = jax.random.split(key, 4)

# Sample collection
n_states_x = 10
n_states_v = 10
gamma = 0.9

# # Weights collection
# n_weights = 500

# # Trainings
# add_infinity = True
# max_bellman_iterations = 20

# ## Linear PBO
# fitting_steps = 2
# training_steps = 200
# batch_size_samples = n_states
# batch_size_weights = n_weights
# learning_rate = {"first": 0.01, "last": 0.001, "duration": training_steps * fitting_steps * n_actions * n_repetitions}

# ## Q-learning
# fitting_steps_q = fitting_steps * training_steps
# learning_rate_q = {"first": 0.01, "last": 0.001, "duration": fitting_steps_q * max_bellman_iterations * n_actions * n_repetitions}

# # Visualisation of errors and performances
# max_bellman_iterations_validation = max_bellman_iterations + 20
# horizon = 10

## Define environment

In [2]:
from pbo.environment.car_on_hill import CarOnHillEnv

max_pos = 1.0
max_velocity = 3.0
n_actions = 2


env = CarOnHillEnv(max_pos, max_velocity, gamma)

## Collect samples

### Samples on the mesh

In [3]:
from pbo.sample_collection.replay_buffer import ReplayBuffer


replay_buffer = ReplayBuffer()

n_samples = n_states_x * n_states_v * n_actions

for state_x in jnp.linspace(-max_pos, max_pos, n_states_x):
    for state_v in jnp.linspace(-max_velocity, max_velocity, n_states_v):
        state = jnp.array([state_x, state_v])
        
        for action in jnp.arange(n_actions):
            env.reset(state)
            next_state, reward, _, _ = env.step(jnp.array([action]))

            replay_buffer.add(state, jnp.array([action]), reward, next_state)

replay_buffer.cast_to_jax_array()

### Visualize samples

In [ ]:
from pbo.sample_collection.count_samples import count_samples
from pbo.utils.state_action_mesh import StateActionMesh


samples_count, n_outside_boxes = count_samples(replay_buffer, states_boxes, actions_boxes)
samples_visu_mesh = StateActionMesh(states, actions, sleeping_time=0)

samples_visu_mesh.set_values(samples_count, zeros_to_nan=True)
samples_visu_mesh.show(
    f"Samples repartition, \n{int(100 * n_outside_boxes / n_samples)}% are outside the box."
)

## Optimal Q function

In [ ]:
optimal_q = env.optimal_Q_mesh()

q_visu_mesh = StateActionMesh(states, actions, 0)

q_visu_mesh.set_values(optimal_q)
q_visu_mesh.show("Optimal q function")

## Train Q with Fitted-Q

In [ ]:
from tqdm.notebook import tqdm

from pbo.sample_collection.dataloader import SampleDataLoader
from pbo.networks.learnable_q import TableQ


data_loader_samples = SampleDataLoader(replay_buffer, batch_size_samples, shuffle_key)
q = TableQ(
    gamma=gamma,
    network_key=q_network_key,
    random_weights_range=None,
    random_weights_key=random_weights_key,
    n_states=n_states,
    n_actions=n_actions,
    learning_rate=learning_rate_q,
    zero_initializer=True
)
validation_initial_weight = q.to_weights(q.params)

training_losses_q = np.zeros(max_bellman_iterations)
validation_losses_q = np.zeros(max_bellman_iterations)
absording_probabilities_q = np.zeros((max_bellman_iterations, horizon))

for bellman_iteration in tqdm(range(max_bellman_iterations)):
    params_target = q.params

    for step in range(fitting_steps_q):
        data_loader_samples.shuffle()
        for batch_samples in data_loader_samples:
            q.params, q.optimizer_state, _ = q.learn_on_batch(q.params, params_target, q.optimizer_state, batch_samples)        

    q_i = q.discretize(q.to_weights(params_target).reshape((-1, q.weights_dimension)), states, actions)[0]
    q_i_plus_1 = q.discretize(q.to_weights(q.params).reshape((-1, q.weights_dimension)), states, actions)[0]
    policy_q = q_i_plus_1.argmax(axis=1)

    training_losses_q[bellman_iteration] = jnp.abs(env.apply_bellman_operator(q_i) - q_i_plus_1).mean()
    validation_losses_q[bellman_iteration] = jnp.abs(optimal_q - q_i_plus_1).mean()
    absording_probabilities_q[bellman_iteration] = env.absorbing_probability(policy_q, horizon)

    print(policy_q)

## Optimal PBO

In [ ]:
from tqdm.notebook import tqdm

from pbo.sample_collection.dataloader import SampleDataLoader
from pbo.networks.learnable_pbo import TabularPBO


data_loader_samples = SampleDataLoader(replay_buffer, batch_size_samples, shuffle_key)

pbo_optimal = TabularPBO(q, max_bellman_iterations, add_infinity, pbo_network_key, learning_rate, n_actions)
pbo_optimal.params["TabularPBONet/linear"]["w"] = gamma * env.transition_proba.T
pbo_optimal.params["TabularPBONet/linear"]["b"] = env.PR.T

training_losses_optimal = np.zeros(max_bellman_iterations_validation)
validation_losses_optimal = np.zeros(max_bellman_iterations_validation)
absording_probabilities_optimal = np.zeros((max_bellman_iterations_validation, horizon))

batch_iterated_weights = validation_initial_weight.reshape((1, -1))
for bellman_iteration in range(max_bellman_iterations_validation):
    q_i = q.discretize(batch_iterated_weights, states, actions)[0]
    batch_iterated_weights = pbo_optimal(pbo_optimal.params, batch_iterated_weights)
    q_i_plus_1 = q.discretize(batch_iterated_weights, states, actions)[0]
    policy_q = q_i_plus_1.argmax(axis=1)

    training_losses_optimal[bellman_iteration] = jnp.abs(env.apply_bellman_operator(q_i) - q_i_plus_1).mean()
    validation_losses_optimal[bellman_iteration] = jnp.abs(optimal_q - q_i_plus_1).mean()
    absording_probabilities_optimal[bellman_iteration] = env.absorbing_probability(policy_q, horizon)

## Collect weights

In [ ]:
from pbo.weights_collection.weights_buffer import WeightsBuffer


weights_buffer = WeightsBuffer()

# Add initial validation weights
weights_buffer.add(validation_initial_weight)

# Add randow weights
q = TableQ(
    gamma=gamma,
    network_key=q_network_key,
    random_weights_range=None,
    random_weights_key=random_weights_key,
    n_states=n_states,
    n_actions=n_actions,
    learning_rate=learning_rate_q
)

while len(weights_buffer) < n_weights:
    weights = q.random_init_weights()
    weights_buffer.add(weights)

weights_buffer.cast_to_jax_array()

## Train non linear PBO

In [ ]:
from tqdm.notebook import tqdm

from pbo.sample_collection.dataloader import SampleDataLoader
from pbo.weights_collection.dataloader import WeightsDataLoader
from pbo.networks.learnable_pbo import TabularPBO


data_loader_samples = SampleDataLoader(replay_buffer, batch_size_samples, shuffle_key)
data_loader_weights = WeightsDataLoader(weights_buffer, batch_size_weights, shuffle_key)
pbo_tabular = TabularPBO(q, max_bellman_iterations, add_infinity, pbo_network_key, learning_rate, n_actions)
importance_iteration = jnp.ones(max_bellman_iterations + 1)
importance_iteration = importance_iteration.at[-1].set(0)

for _ in tqdm(range(training_steps)):
    params_target = pbo_tabular.params

    for _ in range(fitting_steps):
        data_loader_weights.shuffle()
        for batch_weights in data_loader_weights:
            data_loader_samples.shuffle()
            for batch_samples in data_loader_samples:
                pbo_tabular.params, pbo_tabular.optimizer_state, _ = pbo_tabular.learn_on_batch(
                    pbo_tabular.params, params_target, pbo_tabular.optimizer_state, batch_weights, batch_samples, importance_iteration
                )

training_losses_tabular = np.zeros(max_bellman_iterations_validation)
validation_losses_tabular = np.zeros(max_bellman_iterations_validation)
absording_probabilities_tabular = np.zeros((max_bellman_iterations_validation, horizon))

batch_iterated_weights = validation_initial_weight.reshape((1, -1))
for bellman_iteration in range(max_bellman_iterations_validation):
    q_i = q.discretize(batch_iterated_weights, states, actions)[0]
    batch_iterated_weights = pbo_tabular(pbo_tabular.params, batch_iterated_weights)
    q_i_plus_1 = q.discretize(batch_iterated_weights, states, actions)[0]
    policy_q = q_i_plus_1.argmax(axis=1)

    training_losses_tabular[bellman_iteration] = jnp.abs(env.apply_bellman_operator(q_i) - q_i_plus_1).mean()
    validation_losses_tabular[bellman_iteration] = jnp.abs(optimal_q - q_i_plus_1).mean()
    absording_probabilities_tabular[bellman_iteration] = env.absorbing_probability(policy_q, horizon)


## Train linear PBO

In [ ]:
from tqdm.notebook import tqdm

from pbo.sample_collection.dataloader import SampleDataLoader
from pbo.weights_collection.dataloader import WeightsDataLoader
from pbo.networks.learnable_pbo import LinearPBO


data_loader_samples = SampleDataLoader(replay_buffer, batch_size_samples, shuffle_key)
data_loader_weights = WeightsDataLoader(weights_buffer, batch_size_weights, shuffle_key)
pbo = LinearPBO(q, max_bellman_iterations, True, pbo_network_key, learning_rate)  # infinity added
importance_iteration = jnp.ones(max_bellman_iterations + 1)


for _ in tqdm(range(training_steps)):
    params_target = pbo.params
    for _ in range(fitting_steps):
        data_loader_weights.shuffle()
        for batch_weights in data_loader_weights:
            data_loader_samples.shuffle()
            for batch_samples in data_loader_samples:
                pbo.params, pbo.optimizer_state, _ = pbo.learn_on_batch(
                    pbo.params, params_target, pbo.optimizer_state, batch_weights, batch_samples, importance_iteration
                )

training_losses = np.zeros(max_bellman_iterations_validation)
validation_losses = np.zeros(max_bellman_iterations_validation)
absording_probabilities = np.zeros((max_bellman_iterations_validation, horizon))

batch_iterated_weights = validation_initial_weight.reshape((1, -1))
for bellman_iteration in range(max_bellman_iterations_validation):
    q_i = q.discretize(batch_iterated_weights, states, actions)[0]
    batch_iterated_weights = pbo(pbo.params, batch_iterated_weights)
    q_i_plus_1 = q.discretize(batch_iterated_weights, states, actions)[0]
    policy_q = q_i_plus_1.argmax(axis=1)

    training_losses[bellman_iteration] = jnp.abs(env.apply_bellman_operator(q_i) - q_i_plus_1).mean()
    validation_losses[bellman_iteration] = jnp.abs(optimal_q - q_i_plus_1).mean()
    absording_probabilities[bellman_iteration] = env.absorbing_probability(policy_q, horizon)

    print(policy_q)

## Visualize errors in Q functions

In [ ]:
import matplotlib.pyplot as plt 

iterations = range(1, max_bellman_iterations_validation + 1)

plt.plot(iterations, validation_losses, label="pbo linear", color="green")
plt.plot(iterations, validation_losses_tabular, label="pbo max-linear", color="grey", linestyle="--")
plt.plot(iterations, validation_losses_optimal, label="pbo optimal", color="black")
plt.plot(range(1, max_bellman_iterations + 1), validation_losses_q, label="FQI", color="red", linewidth=4)
plt.vlines(max_bellman_iterations, 0, np.max(validation_losses_q), color="black", linestyle="--")

plt.xticks(iterations)
plt.xlabel("iterations")

plt.title(r"$|| Q^* - Q_i ||_1$")
_ = plt.legend()

In [ ]:
plt.plot(iterations, training_losses, label="pbo linear", color="green")
plt.plot(iterations, training_losses_tabular, label="pbo max-linear", color="grey")
plt.plot(iterations, training_losses_optimal, label="pbo optimal", color="black")
plt.plot(range(1, max_bellman_iterations + 1), training_losses_q, label="FQI", color="red")
plt.vlines(max_bellman_iterations, 0, np.maximum(np.max(training_losses_q), np.max(training_losses)), color="black", linestyle="--")

plt.xticks(iterations)
plt.xlabel("iterations")

plt.title(r"$|| \Gamma^*Q_{i-1} - Q_i ||_1$")
_ = plt.legend()

## Understanding the learning process

In [ ]:
empirical_transition_proba = np.zeros((n_states, n_states * n_actions))

for idx_sample in range(len(replay_buffer.states)):
    state = replay_buffer.states[idx_sample, 0]
    action = replay_buffer.actions[idx_sample, 0]
    next_state = replay_buffer.next_states[idx_sample, 0]

    empirical_transition_proba[next_state, state * n_actions + action] += 1 

empirical_transition_proba /= n_repetitions


In [ ]:
np.linalg.norm(empirical_transition_proba - pbo_tabular.params["TabularPBONet/linear"]["w"] / gamma)

## Visualize errors in preformances

In [ ]:
iteration = 5
time = range(1, horizon + 1)


plt.plot(time, absording_probabilities[iteration], label="pbo linear", color="green")
plt.plot(time, absording_probabilities_tabular[iteration], label="pbo max-linear", color="grey", linestyle="--")
plt.plot(time, absording_probabilities_optimal[iteration], label="pbo optimal", color="black")
plt.plot(time, absording_probabilities_q[min([iteration, max_bellman_iterations - 1])], label="FQI", color="red")

plt.xticks(time)
plt.xlabel("time")

plt.title(r"$P^{\pi}(s_i =$ absorbing state)")
_ = plt.legend()

In [ ]:
iteration = 1


plt.plot(iterations, absording_probabilities[:, -1], label="pbo linear", color="green")
plt.plot(iterations, absording_probabilities_tabular[:, -1], label="pbo max-linear", color="grey", linestyle="--")
plt.plot(iterations, absording_probabilities_optimal[:, -1], label="pbo optimal", color="black")
plt.plot(range(1, max_bellman_iterations + 1), absording_probabilities_q[:, -1], label="FQI", color="red")
plt.vlines(max_bellman_iterations, 0, 1, color="black", linestyle="--")

plt.xticks(iterations)
plt.xlabel("iterations")

plt.title(r"$P^{\pi}(s_i =$ absorbing state)" + f" after {horizon} steps")
_ = plt.legend()